# Proyecto 2 - Procesamiento de lenguaje natural
## CommonLit - Evaluate Student Summaries

**Francis Aguilar - 22243**<br>
**Diego García - 22404**<br>
**César López - 22535**<br>
**Ángela García - 22869**


Link del github:<br>
https://github.com/DiegoGarV/Proyecto2-DS

## Planteamiento inicial

1. Situación problemática:<br> 
En el ámbito educativo, la evaluación de resúmenes escritos por estudiantes suele ser un proceso manual, demandante y sujeto a la subjetividad del evaluador. Esto genera dificultades para proporcionar retroalimentación oportuna y consistente, especialmente en contextos con gran número de alumnos. El reto de CommonLit plantea la necesidad de desarrollar métodos automáticos de procesamiento de lenguaje natural (PLN) que apoyen la evaluación objetiva de los resúmenes en cuanto a contenido y calidad de redacción.

2. Problema científico:<br>
¿Es posible construir un modelo de PLN que, a partir del texto de un resumen y su prompt asociado, prediga con precisión las calificaciones de contenido y redacción, utilizando métricas cuantitativas como el MCRMSE para validar su desempeño?

3. Objetivo general:<br>
Desarrollar y evaluar un modelo de procesamiento de lenguaje natural capaz de predecir automáticamente las calificaciones de contenido y redacción en resúmenes estudiantiles, con el fin de apoyar la evaluación educativa de manera objetiva y eficiente.

4. Objetivos específicos:<br>
    i. Analizar el dataset de CommonLit para explorar las características lingüísticas y distribuciones de las calificaciones asociadas a los resúmenes.<br>
    ii. Implementar un sistema de representación textual y entrenar modelos de regresión basados en TF-IDF y algoritmos de aprendizaje supervisado.<br>
    iii. Validar el desempeño de los modelos utilizando validación cruzada y MCRMSE, comparando los resultados obtenidos con diferentes enfoques de modelado.

## Investigación preliminar

Las técnicas más comunes para la detección de patrones en procesamiento de lenguaje natural incluyen métodos estadísticos, representaciones vectoriales y modelos de aprendizaje automático. Entre las que más pueden aportar al proyecto son:

1. Análisis de frecuencia y n-gramas: permiten identificar qué palabras o combinaciones de palabras aparecen con mayor regularidad en los textos, revelando estructuras recurrentes y patrones de estilo.

2. Vectorización de texto (TF-IDF, embeddings): transforman el lenguaje en representaciones numéricas que capturan la importancia relativa de los términos o sus relaciones semánticas.

3. Modelos de clasificación o regresión supervisada: como regresión lineal, máquinas de soporte vectorial o redes neuronales, que aprenden patrones complejos para predecir etiquetas o puntajes a partir de los textos.

Estas técnicas pueden ayudar a extraer características del lenguaje en los resumenes estudiantiles y relacionarlos con las calificaciones de contenido y redacción. Los primeros dos ayudarán a facilitar la captura del vocabulario más característico de los resumenes y el último ayudará a asociar los patrones con el puntaje. De esta manera, se obtiene un sistema capaz de generalizar y evaluar automáticamente nuevos textos de manera objetiva y consistente.

## Análisis inicial del problema y los datos disponibles.

